In [ ]:
import os
from importlib.resources import contents
import json
import codecs

from pypdf import PdfReader, PdfWriter
from dotenv import load_dotenv
import openai
import base64

In [ ]:

INPUT_FOLDER = 'INPUT_FOLDER/'
OUTPUT_FOLDER = 'OUTPUT_FOLDER/'
SIGNATURE_IMAGE = ''
MODEL = "gpt-5"
SYSTEM_MESSAGE = ("""V příloze ti posílám PDF nájemní smlouvy. Doplň mi prosím souřadnice (x,y v pixelech), na které patří podpis nájemníka.
                     Předpokládej standardní velikost stránky (A4) na výšku, předpokládej standardní rozlišení. Odvoď DPI.
                     Formát odpovědi bude JSON:
                     {
                     "page": 2,
                     "x_coordinate": 100,
                     "y_coordinate": 200,
                     "dpi": 100
                     "result" : "ok"
                     }
                     Pokud nebudeš vědět, vrať
                     {
                     "result":"nok",
                     "reason": "why nok"
                     }
                    """)
SYSTEM_MESSAGE_2 = """V příloze ti zísálám jpg soubor.
                      Chci abys i vygeneroval jpeg, kde bude obrázek na následujících souřadnicích níže.
                      Ignoruj "page" atribut. Jedná se standardní PDF A4, a standardní rozlišení.
                      Vrať mi pouze base64 zakodovaný obrázek """

load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
chatgpt_client = openai.OpenAI(api_key=OPENAI_API_KEY)

with open("signature.jpg", "rb") as f:
    img_b64 = base64.b64encode(f.read()).decode('utf-8')



In [ ]:
 def generate_image(json_response):
        completion = chatgpt_client.chat.completions.create(model=MODEL, messages=[{"role": "user", "content": [
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}},
            {
                "type": "text",
                "text": SYSTEM_MESSAGE_2 + str(json_response),
            },
        ]}])
        print(completion.choices[0].message.content)
        image_content = chatgpt_client.files.content(completion.choices[0].message.content)
        with open("test.pdf", "wb") as f:
            f.write(codecs.decode(chatgpt_client.files.content(image_content), "base64"))

In [ ]:

for filename in os.listdir(INPUT_FOLDER):

    contract = chatgpt_client.files.create(
        file=open(f"INPUT_FOLDER/{filename}", "rb"),
        purpose='user_data'
    )

    completion = chatgpt_client.chat.completions.create(model=MODEL, messages= [ { "role":"user",          "content": [
                {
                    "type": "file",
                    "file": {
                        "file_id": contract.id,
                    }
                },
                {
                    "type": "text",
                    "text": SYSTEM_MESSAGE,
                },
            ]}])
    json_response =  json.loads(completion.choices[0].message.content)
    print(json_response)
    if json_response['result'] == 'ok':
        generate_image(json_response)
    else:
        content = None


print(type(content))
